## Interrupt

This notebook assumes knowledge of what double-texting is, which you can learn about in the [double-texting conceptual guide](https://langchain-ai.github.io/langgraph/cloud/concepts/#double-texting).

The guide covers the `interrupt` option for double texting, which interrupts the prior run of the graph and starts a new one with the double-text. This option does not delete the first run, but rather keeps it in the database but sets its status to `interrupted`. Below is a quick example of using the `interrupt` option.

First, let's import our required packages and instantiate our client, assistant, and thread.

In [ ]:
import asyncio

from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client

In [ ]:
client = get_client()
assistant_id = "agent"
thread = await client.threads.create()

In [ ]:
# the first run will be interrupted
interrupted_run = await client.runs.create(
    thread["thread_id"],
    assistant_id,
    input={"messages": [{"role": "human", "content": "what's the weather in sf?"}]},
)
await asyncio.sleep(2)
run = await client.runs.create(
    thread["thread_id"],
    assistant_id,
    input={"messages": [{"role": "human", "content": "what's the weather in nyc?"}]},
    multitask_strategy="interrupt",
)

In [ ]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

We can see that the thread has partial data from the first run + data from the second run

In [ ]:
state = await client.threads.get_state(thread["thread_id"])

In [ ]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

what's the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01MjNtVJwEcpujRGrf3x6Pih', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01MjNtVJwEcpujRGrf3x6Pih)
 Call ID: toolu_01MjNtVJwEcpujRGrf3x6Pih
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/KCASANFR2002/date/2024-6-18", "content": "High 64F. Winds W at 10 to 20 mph. A few clouds from time to time. Low 49F. Winds W at 10 to 20 mph. Temp. San Francisco Weather Forecasts. Weather Underground provides local & long-range weather ..."}]
================================ Human Message =============================

Verify that the original, interrupted run was interrupted

In [ ]:
(await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"]

'interrupted'